#### 这个文档是训练模型专用

In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

#加载数据的方法
def load_dataset(path):
    data = load_files(path)
    print(type(data))
    data_files = np.array(data['filenames'])
    print(data_files[0])
#     data_targets = np_utils.to_categorical(np.array(data['target']), 2)
    data_targets = np.array(data['target'])
    return data_files,data_targets

Using TensorFlow backend.


In [2]:
#加载所有的处理过的猫狗数据
train_files,train_targets=load_dataset('train_set')

<class 'sklearn.utils.Bunch'>
train_set/cat/cat.3557.jpg


In [ ]:
#打印下看看有没有问题
print(train_files[0],train_targets[0])

In [ ]:
#注意,不能直接加载数据集然后再分割,因为这样很占用内存
from sklearn.model_selection import train_test_split

X=train_files
Y=train_targets
x_train,x_valid,y_train,y_valid=train_test_split(X,Y,test_size=0.2,random_state=8888)

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras import Model


base_model=InceptionResNetV2(weights='imagenet',include_top=False)
for layer in base_model.layers:
    layer.trainable=False
    
x=GlobalAveragePooling2D()(base_model.output)
x=Dropout(0.2)(x)
# 添加一个分类器，我们有2个类
predictions=Dense(1,activation='sigmoid')(x)

# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
for i, layer in enumerate(model.layers):
    print(i, layer.name)
    
#输出模型的样式
model.summary()

In [ ]:
#展示模型,因为要安装额外软件,所以linux就注释掉了
#from keras.utils import plot_model
#pip install pydot
#pip install pydot-ng
#pip install graphviz 
#参考https://blog.csdn.net/u011311291/article/details/80298563
#https://packages.ubuntu.com/search?keywords=graphviz&searchon=names
#plot_model(model, to_file='model.png',show_shapes=True)

In [ ]:
#另一种可视化模型的方法
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
#接下来的处理方式是将所有的图片都处理成向量
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input as ir_preprocess_input
from tqdm import tqdm

##pip install  tqdm
#注意要安装tqdm
def path_to_tensor(image_path,target_size=(299,299)):
    '''将图片转换为tensor向量'''
    img=image.load_img(image_path,target_size=target_size)
    x=image.img_to_array(img)
    x=ir_preprocess_input(x)
    return np.expand_dims(x,axis=0)

def paths_to_tensor(image_paths):
    '''批量将图片转换为tensor'''
    list_of_tensor=[path_to_tensor(image_path) for image_path in tqdm(image_paths)]
    return np.vstack(list_of_tensor)
    

In [ ]:
#训练模型看看
from keras.callbacks import ModelCheckpoint
import os
###设置训练模型的epochs的数量
epochs = 30
batch_size=50
check_point_filepath='./saved_models/weights.best.from_scratch_adam.hdf5'
saved_model_path='./saved_models'

#文件夹不存在,就创建文件夹
if not os.path.exists(saved_model_path):
    os.mkdir()
    
####设置检查点
checkpointer = ModelCheckpoint(filepath=check_point_filepath, verbose=1, save_best_only=True)

#设置回调
call_back=[checkpointer]

#加载数据
x_train_data=paths_to_tensor(x_train)
x_valid_data=paths_to_tensor(x_valid)

In [ ]:
#开始训练
epochs=10
history= model.fit(x_train_data,y_train,validation_data=(x_valid_data,y_valid),epochs=epochs,batch_size=batch_size,callbacks=call_back)

In [ ]:
#展示Accuracy数据
def show_acc(history):
    print(history.history.keys())
    figure=plt.figure()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accucary')
    plt.xlabel('accuracy')
    plt.ylabel('epoch')
    #注意!
    plt.legend(['train','valid'],loc='upper left')
    figure.savefig('performance_acc.png')
    

#展示Loss数据  
def show_loss(history):
    print(history.history.keys())
    figure=plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Loss')
    plt.ylabel('epoch')
    #注意!
    plt.legend(['train','valid'],loc='upper left')
    figure.savefig('performance_loss.png')

In [ ]:
# # show_history(history)

# fig = plt.figure()
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='lower left')

# fig.savefig('performance.png')
#展示训练后的数据
show_acc(history)
show_loss(history)

#### 目前遭遇了一个严重问题,在使用数据集5000,epoch=10,batch_size=50的时候,验证集的val_loss:保持在8.2847 ,val_acc:固定在0.4860,令人费解.
#### 我做了如下尝试:
* 1.停止训练,检查load_files,得到的数据,图片和target是一一对应的
* 2.减少了训练集大小和epoch和batch_size来加快训练速度
* 3.修改了model compile时使用的损失函数,从accuracy 改成binary_accuracy



#### 经过查找原因,发现是数据集太小,模型无法收敛了导致的
#### 解决方案:
* 使用云服务器,有aws和gcp等几个可以选择,我使用gcp继续进行模型的训练 

#### 在经过训练后,我又发现了上次类似的问题,验证集数据准确率只有0.51左右.最先考虑的情况是epoch太少,所以选择10-->30,发现作用不大
#### 问题原因:
* 数据集未经过预处理,且预处理的方法是InceptionResNet50独有的,这点需要注意


In [ ]:
# 经过训练.发现上面两图中的Loss在epoch=5的时候就到达了最大.Accurcy差不多是最高的,所以判断epoch<10内模型就应该收敛了,于是重新训练
# 并且将输入尺寸从224 * 224  改成模型默认的299 * 299 希望成绩变好
epochs=10
history= model.fit(x_train_data,y_train,validation_data=(x_valid_data,y_valid),epochs=epochs,batch_size=batch_size,callbacks=call_back)

In [ ]:
# 加载权重,进行fine-tune
model.load_weights(check_point_filepath)
# 记得先编译
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])